In [1]:
using DrWatson

@quickactivate "GRNEvoContingency"

projectname()

"GRNEvoContingency"

In [2]:
include(srcdir("TissueModel_ND.jl"))

include(srcdir("Evolution.jl"))
include(srcdir("FitnessFunctions.jl"))

include(srcdir("NetworkTopologies.jl"))
include(srcdir("Utility.jl"))

# using Plots
using CairoMakie
using Random
using Printf
using DataFrames
using JLD
using StatsPlots
using LinearAlgebra
using NearestNeighbors
using DataInterpolations
using Clustering
using GraphMakie
using Graphs
using ClusterValidityIndices
using MultivariateStats
using KernelDensity
using NetworkLayout
using TravelingSalesmanExact, GLPK

using Memoization
using SparseArrays

using Base.Threads
using Base.Threads: @spawn

using LazySets
using Polyhedra

┌ Warning: Error requiring `UnicodePlots` from `Plots`
│   exception = (LoadError("/Users/boothh/.julia/packages/Plots/qgrW8/src/backends/unicodeplots.jl", 277, LoadError("/Users/boothh/.julia/packages/Plots/qgrW8/src/backends/unicodeplots.jl", 315, UndefVarError(:FileIO))), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000100aa1d1f, Ptr{Nothing} @0x0000000100ad02d3, Ptr{Nothing} @0x0000000100aec3a3, Ptr{Nothing} @0x0000000100aeb283, Ptr{Nothing} @0x0000000100ac21db, Ptr{Nothing} @0x0000000100abf777, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abf6df, Ptr{Nothing} @0x0000000100abffeb, Ptr{Nothing} @0x0000000100aeb7d7, Ptr{Nothing} @0x0000000100aec34b, Ptr{Nothing} @0x0000000100aed0ef, Ptr{Nothing} @0x000000015119c9c3, Ptr{Nothing} @0x0000000100ab7e63, Ptr{Nothing} @0x00000002e9e44377, Ptr{

In [3]:
# run_data = load(datadir("sims/repeated_evolution_different_topologies/a3.jld2"))["raw_data"]; #a3 2500, one mutation
# run_data = load(datadir("sims/repeated_evolution_different_topologies/more_than_one_mut.jld2"))["raw_data"]; #a3 2500, more than one mutation (binomial, 0.2)

# run_data = load(datadir("sims/repeated_evolution_different_topologies/bta_not_inf_single_mut.jld2"))["raw_data"]; #beta 1 single mut

# run_data = load(datadir("sims/repeated_evolution_different_topologies/a8_60cell.jld2"))["raw_data"]; #beta 1 single mut 60 cell

# run_data = load(datadir("sims/repeated_evolution_different_topologies/bta_not_inf_single_mut.jld2"))["raw_data"]; #beta 1 single mut

run_data_cl = load(datadir("sims/repeated_evolution_different_topologies/deletion_prob=0.05_max_gen=50000_mut_prob=0.1_n_target_stripe=1_n_traj=500_noise_cv=0.5_start_network_name=half_right_topology=classical_β=1.0.jld2"))["raw_data"];
run_data_ff = load(datadir("sims/repeated_evolution_different_topologies/deletion_prob=0.05_max_gen=50000_mut_prob=0.1_n_target_stripe=1_n_traj=500_noise_cv=0.5_start_network_name=half_right_topology=feed_forward_β=1.0.jld2"))["raw_data"];
run_data_mi = load(datadir("sims/repeated_evolution_different_topologies/deletion_prob=0.05_max_gen=50000_mut_prob=0.1_n_target_stripe=1_n_traj=500_noise_cv=0.5_start_network_name=half_right_topology=mutual_inh_β=1.0.jld2"))["raw_data"];

In [ ]:
mutable struct LocalLandscape
    origin :: Individual    
    origin_fitness :: Union{Float64,Nothing}
    sample_points :: Union{StepRangeLen,Nothing}
    slice_fitnesses :: Union{Array{Float64, 3},Nothing}
    slice_phenotypes :: Union{Array{Tuple{Float64,Float64}, 3},Nothing}
    transition_prob :: Union{Array{Float64, 2},Nothing}
    debug ::Any
end


function LocalLandscape(start_network::Matrix{Float64},grn_parameters::GRNParameters,development::DESystemSolver)

    p = (start_network,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype)

    origin_fitness,origin_pheno_class = fitness_function(origin.phenotype) 
    
    LocalLandscape(origin,origin_fitness,nothing,nothing,nothing,nothing,nothing)

end

function create_mutant_get_pheno(founder::Individual,development::DESystemSolver,entry::Tuple{Int,Int},step::Float64,fitness_function,noise_application)

    mutant = create_mutant(founder,x->increment_weight(entry,step,x,noise_application),development)

    mutant_fitness,mutant_pheno_class = fitness_function(mutant.phenotype)

    return mutant_fitness

end

function increment_weight(entry::Tuple{Int,Int},step::Float64,w::Matrix{Float64},noise_application)
    new_w = copy(w)
    new_w[entry...] = noise_application(new_w[entry...],step)
    return new_w
end

function compute_slices!(LL::LocalLandscape,range_percentile::Float64,N_sample::Int64,development::DESystemSolver,mutation_op::MutationOperator,fitness_function,noise_application)

    start_stop = quantile.(mutation_op.noise_distribution, [1-range_percentile, range_percentile])

    sample_points = range(start_stop[1],start_stop[2],length = N_sample)

    slice_fitnesses = fill(0.,(size(LL.origin.genotype.p[1])...,N_sample))
    
    @sync for i in 1:size(LL.origin.genotype.p[1],1)
        for j in 1:size(LL.origin.genotype.p[1],2) 
            for s in 1:N_sample
                @spawn slice_fitnesses[i,j,s] = create_mutant_get_pheno(LL.origin,development,(i,j),sample_points[s],fitness_function,noise_application)
            end
        end
    end

    LL.sample_points = sample_points
    LL.slice_fitnesses = slice_fitnesses

end

function calculate_fitness_increase_probability(fitness_slice::Vector{Float64},current_fitness::Float64,sample_points::StepRangeLen,mutation_op::MutationOperator,β::Float64)

    mass = 0.

    dx = step(sample_points)

    for i in 1:length(sample_points)
        Δf = fitness_slice[i] - current_fitness 
        mass+= (cdf(mutation_op.noise_distribution,sample_points[i] + dx/2) - cdf(mutation_op.noise_distribution,sample_points[i] - dx/2))*fixation_probability(Δf,β)
    end

    return mass
end


function calculate_transition_probabilities!(LL::LocalLandscape,mutation_op::MutationOperator,β::Float64)

    prob = mapslices(s->calculate_fitness_increase_probability(s,LL.origin_fitness,LL.sample_points,mutation_op,β),LL.slice_fitnesses,dims = 3)[:,:,1]

    LL.transition_prob = prob 
end

function calculate_transition_probabilities(LL::LocalLandscape,mutation_op::MutationOperator,β::Float64)

    prob = mapslices(s->calculate_fitness_increase_probability(s,LL.origin_fitness,LL.sample_points,mutation_op,β),LL.slice_fitnesses,dims = 3)[:,:,1]

    return prob
end
    
@memoize Dict function LocalLandscape(start_network::Matrix{Float64},range_percentile::Float64,N_sample::Int,grn_parameters::GRNParameters,development::DESystemSolver,mutation_op::MutationOperator,β::Float64,fitness_function,noise_application)

    print("Calculating Loss Landscape...")
    
    LL = LocalLandscape(start_network,grn_parameters,development)

    compute_slices!(LL,range_percentile,N_sample,development,mutation_op,fitness_function,noise_application)

    calculate_transition_probabilities!(LL,mutation_op,β)

    LL

end

function LocalLandscape_mass(start_network::Matrix{Float64},range_percentile::Float64,N_sample::Int,grn_parameters::GRNParameters,development::DESystemSolver,mutation_op::MutationOperator,β::Float64,fitness_function,noise_application)
    
    LL = LocalLandscape(start_network,grn_parameters,development)

    compute_slices!(LL,range_percentile,N_sample,development,mutation_op,fitness_function,noise_application)

    calculate_transition_probabilities!(LL,mutation_op,β)

    LL

end

In [ ]:
function refresh_type(v)
    [i for i in v]
end

mutable struct GenoTrajectories

    fitness_traj :: Vector{Vector{Float64}}
    traj :: Vector{Vector{Matrix{Float64}}}

    geno_traj :: Vector{Matrix{Float64}}

    initial_fitness :: Float64
    initial_genotype :: Vector{Float64}

    full_geno_traj :: Union{Vector{Matrix{Float64}}, Nothing}
    cluster_assignments :: Union{Vector{Int64}, Nothing}
    fgt_data :: Union{Matrix{Float64},Nothing}
    distance_mat ::  Union{Matrix{Float64},Nothing}

    max_clusters :: Union{Int64, Nothing}
    optimal_n_cluster :: Union{Int64, Nothing}
    criterion_values :: Union{Vector{Float64},Nothing}

    debug ::Any
end

function GenoTrajectories(evo_traces::Vector{EvoTrace})

    fitness_traj = map(et->refresh_type(map(x->x[1],et.fitness_trajectory)),evo_traces)
    geno_traj = map(et->reduce(hcat,map(x->vec(x),et.traversed_topologies)),evo_traces);

    traj = map(et->unique(et.traversed_topologies),evo_traces);

    initial_fitness = fitness_traj[1][1]
    initial_genotype = vec(evo_traces[1].traversed_topologies[1])

    GenoTrajectories(fitness_traj,traj,geno_traj,initial_fitness, initial_genotype,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing)

end

function select_marker(edge_value)
    edge_value > 0 ? :ltriangle : :vline
end

function draw_grn(network,color_scheme)

    weight_indices = Tuple.(findall(ones(size(network)) .> 0));

    adjacency_matrix = vcat(network,zeros(1,4))

    ng = SimpleDiGraph(adjacency_matrix)

    edge_indices = [(src(i),dst(i)) for i in edges(ng)]

    edge_values = [round(adjacency_matrix[id...],digits = 3) for id in edge_indices]

    vertex_names = Dict(1=>"A",2=> "B", 3=> "C", 4=> "M")

    vertex_names = [vertex_names[i] for i in vertices(ng)]

    fixed_layout(_) = [(0.5, 1.), (0., 0), (1.,0),(0.5,2.)]

    offsets = [[0.02, 0.0],
                [-0.03, -0.05],
                [0.01, -0.05],
                [0.0, 0.01]]

    e_colors = [color_scheme[findall(x->x==t,weight_indices)[1]] for t in edge_indices]

    f, ax, p = graphplot(ng,layout = fixed_layout,node_color = :black, elabels = string.(edge_values), nlabels = vertex_names,edge_color = e_colors,edge_width = 6.,arrow_size = 30.,arrow_shift = 0.1, arrow_attr = (; marker = [select_marker(edge_values[i]) for i in 1:ne(ng)]))

    p.elabels_rotation[] = Dict(i => edge_indices[i] == (1,4) ? 0.0 : Makie.automatic for i in 1:ne(ng))

    # offsets = 0.05 * (p[:node_pos][] .- p[:node_pos][][1])
    p.nlabels_offset[] = offsets

    autolimits!(ax)
    hidedecorations!(ax); hidespines!(ax)

    f,ax,p
end

function draw_grn!(ax,network,color_scheme)

    weight_indices = Tuple.(findall(ones(size(network)) .> 0));

    adjacency_matrix = vcat(network,zeros(1,4))

    ng = SimpleDiGraph(adjacency_matrix)

    edge_indices = [(src(i),dst(i)) for i in edges(ng)]

    edge_values = [round(adjacency_matrix[id...],digits = 3) for id in edge_indices]

    vertex_names = Dict(1=>"A",2=> "B", 3=> "C", 4=> "M")

    vertex_names = [vertex_names[i] for i in vertices(ng)]

    fixed_layout(_) = [(0.5, 1.), (0., 0), (1.,0),(0.5,2.)]

    offsets = [[0.02, 0.0],
                [-0.075, -0.05],
                [0.025, -0.05],
                [0.0, 0.01]]

    e_colors = [color_scheme[findall(x->x==t,weight_indices)[1]] for t in edge_indices]

    graphplot!(ax,ng,layout = fixed_layout, node_color = :black, elabels = string.(edge_values), nlabels = vertex_names,edge_color = e_colors,edge_width = 6.,arrow_size = 30.,arrow_shift = 0.1, arrow_attr = (; marker = [select_marker(edge_values[i]) for i in 1:ne(ng)]), elabels_rotation = Dict(i => edge_indices[i] == (1,4) ? 0.0 : Makie.automatic for i in 1:ne(ng)),nlabels_offset = offsets)

    autolimits!(ax)
    hidedecorations!(ax); hidespines!(ax)
end

In [ ]:
save_figs = false

In [ ]:
evo_traces_cl = map(x->x[2],run_data_cl);
evo_traces_ff = map(x->x[2],run_data_ff);
evo_traces_mi = map(x->x[2],run_data_mi);

In [ ]:
gt_cl = GenoTrajectories(evo_traces_cl);
gt_ff = GenoTrajectories(evo_traces_ff);
gt_mi = GenoTrajectories(evo_traces_mi);

start_network_cl = reshape(gt_cl.initial_genotype,(3,4))
start_network_ff = reshape(gt_ff.initial_genotype,(3,4))
start_network_mi = reshape(gt_mi.initial_genotype,(3,4))

end_fitness_cl = map(x->x[end],gt_cl.fitness_traj);
end_fitness_ff = map(x->x[end],gt_ff.fitness_traj);
end_fitness_mi = map(x->x[end],gt_mi.fitness_traj);

In [ ]:
# function fitness_route_buckets(gt,n_buckets)
#         all_unique_fitness = unique(reduce(vcat,map(x->unique(x),gt.fitness_traj)))

#         hist_edges = zeros(n_buckets + 1)

#         hist_edges[1] = gt.initial_fitness

#         hist_edges[2:n_buckets] .= LinRange(gt.initial_fitness+eps(),0.9,n_buckets-1) |> collect

#         hist_edges[n_buckets+1] = 1.

#         n_fit_bin = length(hist_edges) - 1

#         h_fitness = fit(Histogram, all_unique_fitness, hist_edges; closed = :left) 

#         return map(traj->map(f->StatsBase.binindex(h_fitness, f),traj),gt.fitness_traj), hist_edges
# end

# function get_last_networks(bin_id,n_bins,gt)

#         binned_fitness_route,h_edges = fitness_route_buckets(gt,n_bins)

#         result = []

#         for (n,bfr) in enumerate(binned_fitness_route)
    
#                 network_ids = findall(x->x == bin_id,bfr)

#                 if length(network_ids) == 0
#                         nothing
#                 else
#                         first_id = minimum(network_ids)
        
#                         last_id = maximum(network_ids)

#                         push!(result,gt.geno_traj[n][:,last_id])
#                 end
#         end
    
#         return result
    
#     end

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,600))
ax = Axis(fig[1,1],xlabel = "Final Fitness",ylabel = "Density", title = "Distribution of end network fitnesses - classical")
CairoMakie.density!(ax,end_fitness_cl)

ax = Axis(fig[1,2],xlabel = "Final Fitness",ylabel = "Density", title = "Distribution of end network fitnesses - feed forward")
CairoMakie.density!(ax,end_fitness_ff)

ax = Axis(fig[1,3],xlabel = "Final Fitness",ylabel = "Density", title = "Distribution of end network fitnesses - mutual inh")
CairoMakie.density!(ax,end_fitness_mi)

if save_figs
    CairoMakie.save(plotsdir("GraphAnalysis","EndFitnesses.png"),fig)
end

fig

In [ ]:
converged_cl = end_fitness_cl .> 0.9

sum(end_fitness_cl .> 0.9)

In [ ]:
converged_ff = end_fitness_ff .> 0.9

sum(end_fitness_ff .> 0.9)

In [ ]:
converged_mi = end_fitness_mi .> 0.9

sum(end_fitness_mi .> 0.9)

In [ ]:
# itp_g = DataInterpolations.LinearInterpolation(unique_geno_visits[:,2],fitness_timestamps);
# full_weight_traj[wi,:] = [itp_g(t) for t in gt.initial_fitness:n:0]

mutable struct InterpolatedLandscape
    origin :: Individual    
    origin_fitness :: Union{Float64,Nothing}
    n_interp :: Union{Int64,Nothing}
    itp_networks :: Union{Array{Float64, 2},Nothing}
    slice_fitnesses :: Union{Array{Float64, 1},Nothing}
    debug ::Any
end

function InterpolatedLandscape(start_network_v::Vector{Float64},end_network_v::Vector{Float64},n_interp::Int64,grn_parameters::GRNParameters,development::DESystemSolver)

    start_network = reshape(start_network_v,(3,4))

    end_network = reshape(end_network_v,(3,4))

    p = (start_network,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype)

    origin_fitness,origin_pheno_class = fitness_function(origin.phenotype) 

    ######

    start_end = hcat(start_network_v,end_network_v)

    itp_networks = zeros(size(start_end,1),n_interp)

    for wi in 1:size(start_end,1)
        itp_g = DataInterpolations.LinearInterpolation(start_end[wi,:],[1,n_interp]);
        itp_networks[wi,:] = [itp_g(t) for t in 1:1:n_interp]
    end

    ######

    slice_fitnesses = zeros(n_interp)

    for (n,network) in enumerate(eachcol(itp_networks))

        if n == 1
            slice_fitnesses[n] = origin_fitness
        else
            new_w = reshape(network,(3,4))
            
            mutant = Individual(remake(origin.genotype, p = (new_w,origin.genotype.p[2:end]...)),development)

            if mutant.phenotype.retcode == :Terminated

                mutant_fitness,mutant_pheno_class = fitness_function(mutant.phenotype)
            else
                mutant_fitness = -1.
            end

            slice_fitnesses[n] = mutant_fitness
        end
    end

    # end_fitness = slice_fitnesses[end]

    # instability = minimum(slice_fitnesses) - 0.5*(origin_fitness+start_fitness)

    InterpolatedLandscape(origin,origin_fitness,n_interp,itp_networks,slice_fitnesses,nothing)

end

In [ ]:
function mutate_method(x,n)

    max_w = 1.
    if x == 0
        proposal = x + n
        r = abs(proposal) > max_w ? max_w*sign(proposal) : proposal
    else
        proposal = x + n*x
        r = abs(proposal) > max_w ? max_w*sign(proposal) : proposal
    end

    return r
end

grn_parameters = DefaultGRNParameters();

development = DefaultGRNSolver()

viable_mutations = ones(Int,Ng,Ng+1)

noise_cv = 0.5

mutation_op = MutationOperator(Normal,(μ = 0.0,σ = noise_cv),findall(viable_mutations .> 0))

n_sample_func() = 1

deletion_prob = 0.05

noise_params = (n_sample_func,deletion_prob)

mutate_function = i -> noise(i,mutation_op,noise_params);

noise_application = (x,n) -> mutate_method(x,n)

output_gene = 3

n_target_stripe = 1

fitness_function = s -> fitness_evaluation(s,x->malt_fitness(x,n_target_stripe),output_gene);

In [ ]:
function instability(network_1,network_2,grn_parameters,development)

    itp_ll = InterpolatedLandscape(network_1,network_2,30,grn_parameters,development);

    return 0.5*(itp_ll.slice_fitnesses[1] + itp_ll.slice_fitnesses[end] + 2) - minimum(itp_ll.slice_fitnesses .+ 1)

end


In [ ]:
end_networks_cl = map(x->x[:,end],gt_cl.geno_traj[converged_cl]);
end_networks_ff = map(x->x[:,end],gt_ff.geno_traj[converged_ff]);
end_networks_mi = map(x->x[:,end],gt_mi.geno_traj[converged_mi]);

all_networks = reduce(vcat,[end_networks_cl,end_networks_ff,end_networks_mi]);

In [ ]:
all_fitness = reduce(vcat,[end_fitness_cl[converged_cl],end_fitness_ff[converged_ff],end_fitness_mi[converged_mi]]);

In [ ]:
label_cl = [1 for n in end_networks_cl]
label_ff = [2 for n in end_networks_ff]
label_mi = [3 for n in end_networks_mi]

all_labels = reduce(vcat,[label_cl,label_ff,label_mi]);

In [ ]:
color_scheme = palette(:tab10)

In [ ]:
# n_sample = length(all_networks)

# dmat = zeros(n_sample,n_sample)

# @sync for i in 1:n_sample
#     for j in 1:n_sample
#         if i>j
#             @spawn dmat[i,j] = instability(all_networks[i],all_networks[j])
#         end
#     end
# end

# for i in 1:n_sample
#     for j in 1:n_sample
#         if i<j
#             dmat[i,j] = dmat[j,i]
#         end
#     end
# end

# for i in 1:n_sample
#     dmat[i,i] = 0
# end

# save(datadir("sims/repeated_evolution_different_topologies","pairwise_instabilities.jld2"),"data",dmat)

In [ ]:
itp_ll_1 = InterpolatedLandscape(end_networks_cl[2],end_networks_cl[5],30,grn_parameters,development);
itp_ll_2 = InterpolatedLandscape(end_networks_cl[3],end_networks_cl[7],30,grn_parameters,development);

In [ ]:
fig = CairoMakie.Figure(resolution = (1000,500), fontsize = 20.)
ax1 = Axis(fig[1,2],title = "Fitness barrier: " * string(0.5*(itp_ll_1.slice_fitnesses[1] + itp_ll_1.slice_fitnesses[end] + 2) - minimum(itp_ll_1.slice_fitnesses .+ 1)), xlabel = L"\lambda", ylabel = L"\text{Fitness, } \mathcal{F}")
ax2 = Axis(fig[1,1],title = "Fitness barrier: " * string(0.5*(itp_ll_2.slice_fitnesses[1] + itp_ll_2.slice_fitnesses[end] + 2) - minimum(itp_ll_2.slice_fitnesses .+ 1)), xlabel = L"\lambda", ylabel = L"\text{Fitness, } \mathcal{F}")

CairoMakie.scatter!(ax1,(0.,itp_ll_1.slice_fitnesses[1]),color = color_scheme[1])
CairoMakie.scatter!(ax2,(0.,itp_ll_2.slice_fitnesses[1]),color = color_scheme[2])

CairoMakie.scatter!(ax1,(1.,itp_ll_1.slice_fitnesses[end]),color = color_scheme[3])
CairoMakie.scatter!(ax2,(1.,itp_ll_2.slice_fitnesses[end]),color = color_scheme[4])

CairoMakie.hlines!(ax1,0.5*(itp_ll_1.slice_fitnesses[1] + itp_ll_1.slice_fitnesses[end]), linestyle = "--", color = color_scheme[9])
CairoMakie.hlines!(ax2,0.5*(itp_ll_2.slice_fitnesses[1] + itp_ll_2.slice_fitnesses[end]), linestyle = "--", color = color_scheme[10])

CairoMakie.lines!(ax1,LinRange(0,1,30),itp_ll_1.slice_fitnesses,color = color_scheme[8])
CairoMakie.lines!(ax2,LinRange(0,1,30),itp_ll_2.slice_fitnesses, color = color_scheme[8])

CairoMakie.linkyaxes!(ax1,ax2)


# L"\mathcal{F}((1-\lambda)W_C + \lambda W_D)"

save_figs = true 

if save_figs
    CairoMakie.save(plotsdir("LMC","LossBarrierExampleClassical.png"),fig)
end

fig

In [ ]:
D_ex = zeros(4,4)

for (i,id_i) in enumerate([3,7,2,5])
    for (j,id_j) in enumerate([3,7,2,5])
        if i>j
            D_ex[i,j] =  instability(end_networks_cl[id_i],end_networks_cl[id_j],grn_parameters,development)
        end
    end
end


In [ ]:
for i in 1:4 
    for j in 1:4
        if i < j
            D_ex[i,j] = D_ex[j,i]
        end
    end
end

In [ ]:
dmat = load(datadir("sims/repeated_evolution_different_topologies","pairwise_instabilities.jld2"))["data"];

In [ ]:
start_net_list = [start_network_cl,start_network_ff,start_network_mi]

SD = zeros(3,3)

for (ni,network_i) in enumerate(start_net_list)
    for (nj,network_j) in enumerate(start_net_list)
        if ni > nj
            SD[ni,nj] = instability(vec(network_i),vec(network_j),grn_parameters,development)
        end
    end
end

In [ ]:
threshold_test = LinRange(0.,1.,20)

max_fitness = 1.9

component_sizes = zeros(length(threshold_test))

for (n,thresh) in enumerate(threshold_test)

    dmat_c = zeros(Int64,size(dmat))

    dmat_c[dmat .< thresh] .= 1

    for i in 1:size(dmat,1)
        dmat_c[i,i] = 0
    end

    g_fit = SimpleGraph(dmat_c)

    con_comp = connected_components(g_fit)

    component_sizes[n] = maximum(map(x->length(x),con_comp)) / length(Graphs.vertices(g_fit))

end


In [ ]:
thresh = 0.1

fig = CairoMakie.Figure(resolution = (800,500), fontsize = 20.)

ax = Axis(fig[1,1], title = "Connectedness versus fitness barrier threshold", xlabel =  L"\text{Connectivity threshold percentage, } \frac{\tau}{b}", ylabel = L"\frac{|C_{\text{max}}|}{|V|}")

CairoMakie.lines!(ax,threshold_test ./ (max_fitness) ,component_sizes,color = color_scheme[2])

ax.xticks = LinRange(0.,1.,11)
ax.xtickformat = x -> string.(round.(x * 100,digits = 0)) .* " %"

ax.yticks = round.(LinRange(0.,1.,11), digits = 2)

hlines!(ax,1., linestyle = "--", color = color_scheme[1])

vlines!(ax,thresh/max_fitness, linestyle = "--", color = color_scheme[3])

if save_figs
    CairoMakie.save(plotsdir("LMC","ComponentSizeVersusThreshold.png"),fig)
end

fig


In [ ]:
dmat_c = zeros(Int64,size(dmat))

dmat_c[dmat .< thresh] .= 1

for i in 1:size(dmat,1)
    dmat_c[i,i] = 0
end


In [ ]:
g_fit = SimpleGraph(dmat_c)

is_connected(g_fit)

In [ ]:
con_comp = connected_components(g_fit)

sort(map(x->length(x),con_comp))

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,1000))
ax = Axis(fig[1,1])

all_colors = [color_scheme[x] for x in all_labels]

graphplot!(ax,g_fit,node_color = all_colors, layout = SFDP(C = 0.5,K = 0.4),node_size = 8.)

hidedecorations!(ax)

if save_figs
    CairoMakie.save(plotsdir("LMC","EntireGraph.png"),fig)
end

fig

In [ ]:
degree_dist = indegree(g_fit)

sum(degree_dist .!= 0.)/length(degree_dist)

In [ ]:
fig = CairoMakie.Figure(resolution = (1100,500), fontsize = 20.)

ax1 = Axis(fig[1,1],title = "Vertex Out Degree Classification (" * string(size(dmat,1)) * " networks)", xticks = (1:2, ["Deg(v) = 0","Deg(v) > 0"]), ylabel = "% of vertices")

CairoMakie.barplot!(ax1,[1,2],[1 - sum(degree_dist .!= 0.)/length(degree_dist),sum(degree_dist .!= 0.)/length(degree_dist)], color = color_scheme[1:2])

ax2 = Axis(fig[1,2],title = "Distribution of Deg(v) > 0", xticks = (1:1, ["Deg(v) > 0"]),ylabel = "Deg(v)")

CairoMakie.boxplot!(ax2,[1 for i in 1:sum(degree_dist .!= 0.)], degree_dist[degree_dist .!= 0.],color = color_scheme[2])

# ax3 = Axis(fig[1,3],title = "Histogram of vertex out-degree distribution (non-zero)")

# CairoMakie.hist!(ax3,degree_dist[degree_dist .!= 0.],color = color_scheme[2])

if save_figs
    CairoMakie.save(plotsdir("LMC","NonZeroConnectedVertices.png"),fig)
end

fig

In [ ]:
max_comp = argmax(map(x->length(x),con_comp))

g_fit_c1 = SimpleGraph(dmat_c[con_comp[max_comp],con_comp[max_comp]])

is_connected(g_fit_c1)

In [ ]:
sum(dmat_c[con_comp[1],con_comp[1]]) / (length(con_comp[1])*length(con_comp[1]))

In [ ]:
global_clustering_coefficient(g_fit_c1)

In [ ]:
minimum(dmat_c)

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,1000))
ax = Axis(fig[1,1],title = "Largest Connected Component - " * string(length(Graphs.vertices(g_fit_c1))) * " vertices")

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 0.5,K = 0.4),node_size = 8.,)

# graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 2.0,K = 0.9),node_size = 8.,)

hidedecorations!(ax)

if save_figs
    CairoMakie.save(plotsdir("LMC","LargestComponent.png"),fig)
end


fig

In [ ]:
max_clique = maximal_cliques(g_fit_c1)

max_id = argmax(map(x->length(x),max_clique))

In [ ]:
cliques_3 = findall(x->x==3,map(x->length(x),max_clique));

In [ ]:
n_c3 = length(cliques_3) 

clique_conn = zeros(n_c3,n_c3)

for i in 1:n_c3
    for j in 1:n_c3

        if i != j
            clique_conn[i,j] = length(Set(max_clique[cliques_3[j]]) ∩ Set(max_clique[cliques_3[i]]))
        end
    end
end

In [ ]:
disjoint_3 = findall(x->x==0,clique_conn)

In [ ]:
cliques_3[21]

In [ ]:
# max_id = 2701

# max_id_1 = 139775
# max_id_2 = 2544362

# max_id_1 = 2730504
# max_id_2 = 2544362

max_id_1 = 2700
max_id_2 = 139766

max_id_1 = 2806674
max_id_2 = 139776

max_id_1 = 2752231
max_id_2 = 2752229

dis_id_1 = 2806664
dis_id_2 = 2785215

# sample_12 = StatsBase.sample(disjoint_3,1)[1]

# max_id_1 = cliques_3[sample_12[1]]
# max_id_2 = cliques_3[sample_12[2]]

# core_id_1 = con_comp[max_comp][max_clique[max_id_2]]
# core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_1)

# core_id_1 = con_comp[max_comp][max_clique[max_id_1]]
# core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_1)


In [ ]:
# using DisjointCliqueCover

# using SimpleGraphAlgorithms
# using SimpleGraphConverter

In [ ]:
# chromatic_number(SimpleGraphConverter.UG(g_fit_c1))

In [ ]:
save_figs = true

In [ ]:
core_id_1 = con_comp[max_comp][max_clique[max_id_1]]

core_g_1 = SimpleGraph(dmat_c[core_id_1,core_id_1])

core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_1)

# core_colour = map(x->c_label_top[x],max_clique[max_id])

fig = CairoMakie.Figure(resolution = (500,500))
ax = Axis(fig[1,1],title = "Example " * string(length(Graphs.vertices(core_g_1))) * " vertex clique")

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,core_g_1, node_color = core_colour_1,layout = Spring(),node_size = 20.)

# graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 2.0,K = 0.9),node_size = 8.,)

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","3-vertex-clique.png"),fig)
end

hidedecorations!(ax)

fig

In [ ]:
# conn_core = k_core(g_fit_c1)

# core_id = con_comp[max_comp][conn_core]


# core_colour = map(x->c_label_top[x],max_clique[max_id])

fig = CairoMakie.Figure(resolution = (1000,500))
ax1 = Axis(fig[1,1],title = "Example of two 3-vertex cliques which share 2 vertices")

core_id_both = con_comp[max_comp][unique(vcat(max_clique[max_id_1],max_clique[max_id_2]))]
core_g_1 = SimpleGraph(dmat_c[core_id_both,core_id_both])

core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_both)

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax1,core_g_1, node_color = core_colour_1,layout = Spring(),node_size = 20.)

ax2 = Axis(fig[1,2],title = "Example of two 3-vertex cliques with disjoint vertices")

core_id_both = con_comp[max_comp][unique(vcat(max_clique[dis_id_1],max_clique[dis_id_2]))]
core_g_1 = SimpleGraph(dmat_c[core_id_both,core_id_both])

core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_both)

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax2,core_g_1, node_color = core_colour_1,layout = Spring(),node_size = 20.)

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","two-3-vertex-clique-disjoint.png"),fig)
end

# graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 2.0,K = 0.9),node_size = 8.,)

hidedecorations!(ax1)

hidedecorations!(ax2)

fig

In [ ]:
using LazySets
using Polyhedra

In [ ]:
clique_hull_1 = convex_hull(all_networks[core_id_1]);

P1 = VPolytope(clique_hull_1);

In [ ]:
x = sum(clique_hull_1)/length(clique_hull_1);

In [ ]:
x ∈ P1

In [ ]:
test_mat = reshape(x,(3,4))

In [ ]:
p = (test_mat,grn_parameters.degradation)

genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
phenotype  = solve(genotype,development.alg;development.kwargs...)

origin = Individual(genotype,phenotype);

print(fitness_function(origin.phenotype))

In [ ]:
print(fitness_function(origin.phenotype))

CairoMakie.lines(origin.phenotype.u[end][3,:])

In [ ]:
n_sample = 1000

hull_sample_results_1 = []

ch_geno_1 = []

for _ in 1:n_sample
    a_coeff = rand(length(clique_hull_1))

    a_coeff = a_coeff ./ sum(a_coeff)

    test_genotype = sum([clique_hull_1[i]*a_coeff[i] for i in 1:length(clique_hull_1)])

    push!(ch_geno_1,test_genotype)

    test_genotype_mat = reshape(test_genotype,(3,4))

    p = (test_genotype_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(hull_sample_results_1,fitness_function(origin.phenotype)[1])
end

In [ ]:
function pairplot_makie_ass(X,extreme_networks,ass,colorscheme)

    fig = CairoMakie.Figure(resolution = (5000,5000),fontsize = 40.)

    vertex_names = Dict(1=>"A",2=> "B", 3=> "C", 4=> "M")

    weight_indices = Tuple.(findall(ones(3,4) .> 0));

    weight_names = [string(vertex_names[last(t)]) * "=>" * string(vertex_names[first(t)]) for t in weight_indices]

    color_scheme = palette(:tab10)

    grid_entries = Tuple.(findall(x->x > 0, ones(size(X,1),size(X,1))))

    for entry in grid_entries
        # if (entry[1] <= 12) && (entry[2] <= 12)
        if entry[1] == entry[2]
            ax = Axis(fig[entry...], xlabel = weight_names[entry[2]],ylabel = weight_names[entry[1]])
            CairoMakie.density!(ax,X[entry[2],:])

            if entry[2] != 1
                hideydecorations!(ax)
            end
        
            if (entry[1] != length(weight_indices))
                hidexdecorations!(ax)
            end

        elseif entry[1] >= entry[2]
            ax = Axis(fig[entry...], xlabel = weight_names[entry[2]],ylabel = weight_names[entry[1]])
            CairoMakie.scatter!(ax,X[[entry[2],entry[1]],:],color = ass,colormap  = colorscheme)
            CairoMakie.scatter!(ax,extreme_networks[[entry[2],entry[1]],:],color = :red,markersize = 20.)

            if entry[2] != 1
                hideydecorations!(ax)
            end
        
            if (entry[1] != length(weight_indices))
                hidexdecorations!(ax)
            end
            
        else
            nothing
        end

    end

    rowgap!(fig.layout, 2.)
    colgap!(fig.layout, 4.)

    return fig

end

In [ ]:
ch_X_1 = reduce(hcat,ch_geno_1);
all_label_X1 = [1 for i in 1:size(ch_X_1,2)]
extreme_net_1 = reduce(hcat,clique_hull_1)

In [ ]:
fig = pairplot_makie_ass(ch_X_1,extreme_net_1,all_label_X1,[:black])

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","cliqueA_pairplot.png"),fig)
end

fig
# pairplot_makie_ass(ch_X_1,extreme_net_1,refresh_type(hull_sample_results_1),:thermal)

In [ ]:
core_id_2 = con_comp[max_comp][max_clique[max_id_2]]

clique_hull_2 = convex_hull(all_networks[core_id_2]);

P2 = VPolytope(clique_hull_2);

n_sample = 1000

hull_sample_results_2 = []

ch_geno_2 = []

for _ in 1:n_sample
    a_coeff = rand(length(clique_hull_2))

    a_coeff = a_coeff ./ sum(a_coeff)

    test_genotype = sum([clique_hull_2[i]*a_coeff[i] for i in 1:length(clique_hull_2)])

    push!(ch_geno_2,test_genotype)

    test_genotype_mat = reshape(test_genotype,(3,4))

    p = (test_genotype_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(hull_sample_results_2,fitness_function(origin.phenotype)[1])
end

In [ ]:
ch_X_2 = reduce(hcat,ch_geno_2);
all_label_X2 = [2 for i in 1:size(ch_X_2,2)]
extreme_net_2 = reduce(hcat,clique_hull_2)

In [ ]:
fig = pairplot_makie_ass(ch_X_2,extreme_net_2,all_label_X2,[:green])

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","cliqueB_pairplot.png"),fig)
end

fig

In [ ]:
ch_X_12 = hcat(ch_X_1,ch_X_2)

all_label_X12 = vcat(all_label_X1,all_label_X2);
fig = pairplot_makie_ass(ch_X_12,extreme_net_2,all_label_X12,[:green,:black])

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","cliqueAB_pairplot.png"),fig)
end

fig

# all_label_X12 = vcat(refresh_type(hull_sample_results_1),refresh_type(hull_sample_results_2));
# pairplot_makie_ass(ch_X_12,extreme_net_2,all_label_X12,:thermal)

In [ ]:

enet1_fitness = []

for m in eachcol(extreme_net_1)

    test_mat = reshape(m,(3,4))

    p = (test_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(enet1_fitness,fitness_function(origin.phenotype)[1])
end

enet2_fitness = []

for m in eachcol(extreme_net_2)

    test_mat = reshape(m,(3,4))
    
    p = (test_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(enet2_fitness,fitness_function(origin.phenotype)[1])
end

In [ ]:

fig = CairoMakie.Figure(resolution = (1500,500), fontsize = 20.)

ax1 = Axis(fig[1,1],title = "Fitness of 1000 networks sampled from clique A simplex", xticks = (1:1, ["Simplex A"]),ylabel = "Fitness")

CairoMakie.boxplot!(ax1,[1 for i in 1:length(hull_sample_results_1)],refresh_type(hull_sample_results_1))

hlines!(ax1,mean(enet1_fitness),linestyle = "--", color = :red)

hlines!(ax1,mean(enet1_fitness) - 2*thresh,linestyle = "--", color = :purple)

ax2 = Axis(fig[1,2],title = "Fitness of 1000 networks sampled from clique B simplex", xticks = (1:1, ["Simplex B"]),ylabel = "Fitness")

CairoMakie.boxplot!(ax2,[1 for i in 1:length(hull_sample_results_2)],refresh_type(hull_sample_results_2))

hlines!(ax2,mean(enet2_fitness),linestyle = "--", color = :red)

hlines!(ax2,mean(enet2_fitness) - 2*thresh,linestyle = "--", color = :purple)

linkyaxes!(ax1,ax2)

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","cliqueAB_fitness.png"),fig)
end



fig

In [ ]:
core_id_1 = con_comp[max_comp][max_clique[dis_id_1]]

clique_hull_1 = convex_hull(all_networks[core_id_1]);

P2 = VPolytope(clique_hull_1);

n_sample = 1000

hull_sample_results_1 = []

ch_geno_1 = []

for _ in 1:n_sample
    a_coeff = rand(length(clique_hull_1))

    a_coeff = a_coeff ./ sum(a_coeff)

    test_genotype = sum([clique_hull_1[i]*a_coeff[i] for i in 1:length(clique_hull_1)])

    push!(ch_geno_1,test_genotype)

    test_genotype_mat = reshape(test_genotype,(3,4))

    p = (test_genotype_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(hull_sample_results_1,fitness_function(origin.phenotype)[1])
end

ch_X_1 = reduce(hcat,ch_geno_1);
all_label_X1 = [1 for i in 1:size(ch_X_1,2)]
extreme_net_1 = reduce(hcat,clique_hull_1)

core_id_2 = con_comp[max_comp][max_clique[dis_id_2]]

clique_hull_2 = convex_hull(all_networks[core_id_2]);

P2 = VPolytope(clique_hull_2);

n_sample = 1000

hull_sample_results_2 = []

ch_geno_2 = []

for _ in 1:n_sample
    a_coeff = rand(length(clique_hull_2))

    a_coeff = a_coeff ./ sum(a_coeff)

    test_genotype = sum([clique_hull_2[i]*a_coeff[i] for i in 1:length(clique_hull_2)])

    push!(ch_geno_2,test_genotype)

    test_genotype_mat = reshape(test_genotype,(3,4))

    p = (test_genotype_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(hull_sample_results_2,fitness_function(origin.phenotype)[1])
end

ch_X_2 = reduce(hcat,ch_geno_2);
all_label_X2 = [2 for i in 1:size(ch_X_2,2)]
extreme_net_2 = reduce(hcat,clique_hull_2)

In [ ]:
ch_X_12 = hcat(ch_X_1,ch_X_2)

all_label_X12 = vcat(all_label_X1,all_label_X2);
fig = pairplot_makie_ass(ch_X_12,extreme_net_2,all_label_X12,[:orange,:green])

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","DisjointcliqueAB_pairplot.png"),fig)
end

fig


In [ ]:
core_id_1 = con_comp[max_comp][max_clique[max_id]]

core_g_1 = SimpleGraph(dmat_c[core_id_1,core_id_1])

core_colour_1 = map(x->color_scheme[all_labels[x]],core_id_1)

# core_colour = map(x->c_label_top[x],max_clique[max_id])

fig = CairoMakie.Figure(resolution = (1000,1000))
ax = Axis(fig[1,1],title = "Example " * string(length(Graphs.vertices(core_g_1))) * " vertex clique")

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,core_g_1, node_color = core_colour_1,layout = Spring(),node_size = 20.)

# graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 2.0,K = 0.9),node_size = 8.,)

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","max-clique.png"),fig)
end

hidedecorations!(ax)

fig

In [ ]:
clique_hull_1 = convex_hull(all_networks[core_id_1]);

P1 = VPolytope(clique_hull_1);

In [ ]:
n_sample = 1000

hull_sample_results_1 = []

ch_geno_1 = []

for _ in 1:n_sample
    a_coeff = rand(length(clique_hull_1))

    a_coeff = a_coeff ./ sum(a_coeff)

    test_genotype = sum([clique_hull_1[i]*a_coeff[i] for i in 1:length(clique_hull_1)])

    push!(ch_geno_1,test_genotype)

    test_genotype_mat = reshape(test_genotype,(3,4))

    p = (test_genotype_mat,grn_parameters.degradation)

    genotype = ODEProblem(gene_regulation_1d!,grn_parameters.g0,(0,Inf),p)
    phenotype  = solve(genotype,development.alg;development.kwargs...)

    origin = Individual(genotype,phenotype);

    push!(hull_sample_results_1,fitness_function(origin.phenotype)[1])
end

In [ ]:
fig = CairoMakie.Figure(resolution = (1500,500), fontsize = 20.)

ax1 = Axis(fig[1,1],title = "Fitness of 1000 networks sampled from max clique simplex", xticks = (1:1, ["Simplex A"]),ylabel = "Fitness")

CairoMakie.boxplot!(ax1,[1 for i in 1:length(hull_sample_results_1)],refresh_type(hull_sample_results_1))

hlines!(ax1,mean(enet1_fitness),linestyle = "--", color = :red)

hlines!(ax1,mean(enet1_fitness) - 2*thresh,linestyle = "--", color = :purple)

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","maxclique_fitness.png"),fig)
end

fig

In [ ]:
ch_X_1 = reduce(hcat,ch_geno_1);
all_label_X1 = [1 for i in 1:size(ch_X_1,2)]
extreme_net_1 = reduce(hcat,clique_hull_1);

In [ ]:
fig = pairplot_makie_ass(ch_X_1,extreme_net_1,all_label_X1,[:blue])

if save_figs
    CairoMakie.save(plotsdir("LMC_Explanation","maxclique_pairplot.png"),fig)
end

fig

In [ ]:
cliques_4 = findall(x->x == 4 ,map(x->length(x),max_clique))

n_c4 = length(cliques_4) 

clique_conn = zeros(n_c4,n_c4)

for i in 1:n_c4
    for j in 1:n_c4

        if i != j

            core_id_i = con_comp[max_comp][max_clique[cliques_4[i]]]
            core_id_j = con_comp[max_comp][max_clique[cliques_4[j]]]

            clique_hull_i = convex_hull(all_networks[core_id_i]);

            clique_hull_j = convex_hull(all_networks[core_id_j]);

            Pi =  VPolytope(clique_hull_i);
            Pj =  VPolytope(clique_hull_j);
            
            clique_conn[i,j] = 1. - Float64(isempty(intersect(Pi,Pj))) # if they intersect (non empty = false) then they are connected

            # clique_conn[i,j] = Pi ⊆ Pj # if they intersect (non empty = false) then they are connected

            # clique_conn[j,i] = Pj ⊆ Pi # if they intersect (non empty = false) then they are connected
        end
    end
end



In [ ]:
cliques_4

In [ ]:
# clique_g_4 = SimpleGraph(clique_conn)

# # core_colour = map(x->c_label_top[x],max_clique[max_id])

# fig = CairoMakie.Figure(resolution = (2000,1000))
# ax = Axis(fig[1,1],title = "Example clique - " * string(length(Graphs.vertices(core_g))) * " vertices")

# # all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

# graphplot!(ax,clique_g_4,layout = Spring(),node_size = 20.,)

# fig

In [405]:
v_list_cliques = []

for mi in max_clique
    
    core_id_1 = con_comp[max_comp][mi]

    clique_hull_1 = convex_hull(all_networks[core_id_1]);

    P1 = VPolytope(clique_hull_1);

    v_list = []

    for v in all_networks
        if v ∈ P1 && !(v ∈ all_networks[core_id_1])
            push!(v_list,v)
        end
    end

    push!(v_list_cliques,v_list)
end

In [403]:
core_id_1 = con_comp[max_comp][max_clique[max_id]]

2860421-element Vector{Vector{Int64}}:
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  154, 182, 119, 125, 46, 87, 4, 54, 161, 88]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  166, 162, 154, 182, 119, 125, 46, 87, 4, 36]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  154, 182, 119, 125, 46, 87, 90, 161, 54, 88]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  162, 154, 182, 119, 125, 46, 87, 90, 161, 98]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  154, 182, 119, 125, 46, 87, 90, 36, 9, 98]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  178, 46, 70, 50, 87, 90, 98, 76, 101, 161]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  46, 70, 50, 87, 90, 98, 76, 101, 36, 9]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  46, 70, 50, 87, 90, 144, 76, 88, 54, 161]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  46, 70, 50, 87, 90, 144, 76, 101, 54, 161]
 [185, 179, 147, 116, 3, 15, 35, 114, 21, 13  …  46, 70, 50, 87, 90, 144, 76, 101, 36, 9]
 ⋮
 [459, 29, 498, 290, 531, 94, 541, 3

In [401]:
clique_hull_1

3-element Vector{Vector{Float64}}:
 [0.0, 0.09693796878733349, 0.0374282813868107, 0.0, 0.08295322241092491, 0.06668291644535018, 0.0, -0.004014408753028725, 0.6146272196396064, 0.4036071113269578, 0.0, -0.5015019275667463]
 [-0.017400509561446516, 0.09693796878733349, 0.031137024397954915, 0.4332070576923477, 0.0, 0.0794130841270203, 0.0, -0.3160893033847306, 0.42984467203949484, 0.24162509598626997, 0.0, -0.5069012673596436]
 [0.0, 0.1666738767506181, 0.04018530796833075, 0.061882132501777154, 0.0, 0.0, -0.005339414010470165, 0.0, 0.456606772665309, 0.3543212133922646, 0.0, -0.2049683259522448]

In [402]:
v_list

Any[]

In [390]:
extreme_net_1 \ all_networks[1]

3-element Vector{Float64}:
 -0.005401960137900122
 -0.19587547235458339
  0.7179056157527245

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,1000))
ax = Axis(fig[1,1])

# color_scheme = palette(:tab20)

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,g_fit_c1, node_color = [color_scheme[x] for x in label_propagation(g_fit_c1)[1]],layout = SFDP(C = 0.5,K = 0.4),node_size = 8.)

hidedecorations!(ax)

if save_figs
    CairoMakie.save(plotsdir("LMC","LargestComponentLabelPrediction.png"),fig)
end


fig

In [ ]:
ground_truth_c1 = all_labels[con_comp[1]]

prediction_c1 = label_propagation(g_fit_c1)[1];

In [ ]:
using MLJ

In [ ]:
counts(ground_truth_c1,prediction_c1)

In [ ]:
accuracy(prediction_c1,ground_truth_c1)

In [ ]:
misclassification_rate(prediction_c1,ground_truth_c1)

In [ ]:
hc = hclust(dmat[con_comp[1],con_comp[1]], linkage = :single)

In [ ]:
StatsPlots.plot(hc, size=(2000,1000))

In [ ]:
id_c1_cl = findall(x->x==1,all_labels) ∩ con_comp[1]
id_c1_ff = findall(x->x==2,all_labels) ∩ con_comp[1]
id_c1_mi = findall(x->x==3,all_labels) ∩ con_comp[1];

In [ ]:
g_fit_c1_cl = SimpleGraph(dmat_c[id_c1_cl,id_c1_cl])

is_connected(g_fit_c1_cl)

In [ ]:
g_fit_c1_ff = SimpleGraph(dmat_c[id_c1_ff,id_c1_ff])

is_connected(g_fit_c1_ff)

In [ ]:
g_fit_c1_mi = SimpleGraph(dmat_c[id_c1_mi,id_c1_mi])

is_connected(g_fit_c1_mi)

In [ ]:
# fig = CairoMakie.Figure(resolution = (2000,2000))
# ax = Axis(fig[1,1])

# # all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

# graphplot!(ax,g_fit_c1_mi, node_color = color_scheme[2],layout = SFDP(C = 0.5,K = 0.4),node_size = 8.)

# fig

In [ ]:
c_label_top = map(x->palette(:tab20)[sum(x .!= 0.)], all_networks);

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,2000))
ax = Axis(fig[1,1])

all_colors = [color_scheme[x] for x in all_labels]

graphplot!(ax,g_fit,node_color = c_label_top , layout = SFDP(C = 0.5,K = 0.4),node_size = [10*x for x in all_fitness])

hidedecorations!(ax)

fig

In [ ]:
fig = CairoMakie.Figure(resolution = (2000,2000))
ax = Axis(fig[1,1])

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,g_fit_c1, node_color = [c_label_top[i] for i in con_comp[1]],layout = SFDP(C = 0.5,K = 0.4),node_size = 8.,)


# graphplot!(ax,g_fit_c1, node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = SFDP(C = 2.0,K = 0.9),node_size = 8.,)

hidedecorations!(ax)

fig

In [ ]:
n_conn =  map(x->sum(x .!= 0.), all_networks);

In [ ]:
CairoMakie.boxplot(n_conn,indegree(g_fit))

In [ ]:
mst,weights = boruvka_mst(g_fit_c1,dmat[con_comp[1],con_comp[1]]; minimize=true);

In [ ]:
edge_weights = []

for e in mst
    push!(edge_weights,dmat[con_comp[1],con_comp[1]][src(e),dst(e)])
end

In [ ]:
# CairoMakie.hist([(i/2.0)*100 for i in edge_weights])

# thresh = 0.1

fig = CairoMakie.Figure(resolution = (800,500), fontsize = 20.)
ax = Axis(fig[1,1], title = "Distribution of edge weights (fitness barriers) in the minimal spanning tree", xlabel =  L"\frac{D_{i,j}}{b} \text{   s.t. } e_{i\rightarrow j} \in MST(E) ", ylabel = "Number of edges")

CairoMakie.hist!(ax,edge_weights ./ (max_fitness),color = color_scheme[2])

vlines!(ax,mean(edge_weights), linestyle = "--")

# ax.xticks = threshold_test[1:1:20] 
ax.xtickformat = x -> string.(round.(x * 100,digits = 0)) .* " %"

if save_figs
    CairoMakie.save(plotsdir("LMC","MinimalSpanningEdgeWeights.png"),fig)
end

fig


In [ ]:
mean(edge_weights)

In [ ]:
mst_g = SimpleGraph(mst)

fig = CairoMakie.Figure(resolution = (2000,2000), fontsize = 20.)

ax = Axis(fig[1,1], title = "Minimal Spanning Tree of Largest Connected Component")

# all_colors = [x == 1 ? :red : x == 2 ? :blue : :green for x in all_labels]

graphplot!(ax,mst_g,node_color = [color_scheme[all_labels[i]] for i in con_comp[1]],layout = Stress(),node_size = 12.,)

hidedecorations!(ax)

if save_figs
    CairoMakie.save(plotsdir("LMC","MinimalSpanningTree_Sq.png"),fig)
end

fig

In [ ]:
mst_vector = []
for e in mst
    push!(mst_vector,all_networks[con_comp[1]][src(e)] .- all_networks[con_comp[1]][dst(e)])
end

In [ ]:
mst_mat = reduce(hcat,mst_vector)

In [ ]:
using RowEchelon

In [ ]:
_, pivots = rref_with_pivots(mst_mat)
mst_mat[:, pivots]